In [101]:
import networkx as nx
import pandas
from hatchet import *
import os


In [102]:
callflow_path = "/home/vidi/Suraj/llnl/CallFlow/"
dataset_path = ["./data/hpctoolkit-lulesh-2"]

In [103]:
# Create Graphframes.

def create_gfs(file_format, paths):    
        print("Creating graphframes....")                                                                                             
        ret = []                                                                                                                         
        for idx, path in enumerate(paths):
            path = os.path.abspath(os.path.join(callflow_path, path)) 
            gf = GraphFrame()   
            gf.from_hpctoolkit(path, 3)                                                                            
            ret.append(gf) 
            print(str(idx) + ":" + path)                                                                                              
            print("[Create] Rows in dataframe: {0}".format(gf.dataframe.shape[0]))                       
        return ret 

gfs = create_gfs('hpctoolkit', dataset_path)

Creating graphframes....
aaaaaaaaaaaa                                 CPUTIME (usec) (I)  CPUTIME (usec) (E)   nid  \
node                      rank                                                 
<program root>            0.0            4612081.0                 0.0     2   
                          1.0            4614204.0                 0.0     2   
                          2.0            4620810.0                 0.0     2   
                          3.0            4615792.0                 0.0     2   
                          4.0            4615343.0                 0.0     2   
                          5.0            4613806.0                 0.0     2   
                          6.0            4614665.0                 0.0     2   
                          7.0            4616159.0                 0.0     2   
main                      0.0            4612081.0                 0.0     4   
                          1.0            4614204.0                 0.0     4   
  

In [104]:
# util functions

def lookup(df, node):                                                                                                                    
    return df.loc[df['node'] == node] 

def getMaxIncTime(gf):                                                                                                                   
    ret = 0.0                                                                                                                            
    for root in gf.graph.roots:                                                                                                          
        ret = max(ret, lookup(gf.dataframe, root)['CPUTIME (usec) (I)'].max())                                                           
    return ret                                                                                                                           
                                                                                                                                         
def getMaxExcTime(gf):                                                                                                                   
    ret  = gf.dataframe['CPUTIME (usec) (E)'].max()                                                                                      
    return ret                                                                                                                           
               
def special_lookup(gf, df_index):   
    return gf.dataframe.loc[gf.dataframe['name'] == df_index] 

In [105]:
# Filter graphframe and graph

  def filter_gfs(gfs, filterBy):                                                                                                   
        # Create the graph frames from the paths and corresponding format using hatchet                                                  
        fgfs = []                                                                                                                        
        # Filter graphframes based on threshold                                                                                          
        for idx, gf in enumerate(gfs):                                                                                              
            print("Filtering the dataframe!")                                                                                         
            if filterBy == "IncTime":                                                                                          
                max_inclusive_time = getMaxIncTime(gf)                                                                             
                filter_gf = gf.filter(lambda x: True if(x['CPUTIME (usec) (I)'] > 0.01*max_inclusive_time) else False)                   
            elif self.args.filterBy == "ExcTime":                                                                                        
                max_exclusive_time = getMaxExcTime(gf)                                                                             
                print('[Filter] By Exclusive time = {0})'.format(max_exclusive_time))                                                 
                filter_gf = gf.filter(lambda x: True if (x['CPUTIME (usec) (E)'] > 0.01*max_exclusive_time) else False)                  
            else:                                                                                                                        
                print("Not filtering.... Can take forever. Thou were warned")                                                         
                filter_gf = gf                                                                                                           
            print('[Filter] Removed {0} rows.)'.format(gf.dataframe.shape[0] - filter_gf.dataframe.shape[0]))                                                                                                                            
            print("Grafting the graph!")                                                                                            
            filter_gf = filter_gf.graft()                                                                                                
            print("[Graft] {0} rows left".format(filter_gf.dataframe.shape[0]))                           
            fgfs.append(filter_gf)                                                                                                       
        return fgfs      

fgfs = filter_gfs(gfs, 'IncTime')
print(fgfs[0].dataframe)

IndentationError: unexpected indent (<ipython-input-105-65abcfb87935>, line 3)

In [106]:
# add df_index to a graph frame.
def df_index(gf):
    gf.dataframe['n_index'] = gf.dataframe.groupby('nid').ngroup()
    print(gf.dataframe['n_index'])
df_index(fgfs[0])

node                       rank
<program root>             0.0       0
                           1.0       0
                           2.0       0
                           3.0       0
                           4.0       0
                           5.0       0
                           6.0       0
                           7.0       0
main                       0.0       1
                           1.0       1
                           2.0       1
                           3.0       1
                           4.0       1
                           5.0       1
                           6.0       1
                           7.0       1
Loop@lulesh.cc:2771        0.0       2
                           1.0       2
                           2.0       2
                           3.0       2
                           4.0       2
                           5.0       2
                           6.0       2
                           7.0       2
TimeIncrement(Domain&)     1.0  

In [107]:
#testing state.lookup

def node_hash_mapper(df):                                                                                                             
    ret = {}                                                                                                                         
    for idx, row in df.iterrows():                                                                                              
        df_node_index = str(row.node.df_index)                                                                                       
        row['df_index'] = df_node_index
    return ret  

hash = node_hash_mapper(fgfs[0].dataframe)
print(fgfs[0].dataframe['df_index'])
print("total nodes in graph: {0}".format(len(hash.keys())))
print(hash)

node                       rank
<program root>             0.0       0
                           1.0       0
                           2.0       0
                           3.0       0
                           4.0       0
                           5.0       0
                           6.0       0
                           7.0       0
main                       0.0       1
                           1.0       1
                           2.0       1
                           3.0       1
                           4.0       1
                           5.0       1
                           6.0       1
                           7.0       1
Loop@lulesh.cc:2771        0.0       2
                           1.0       2
                           2.0       2
                           3.0       2
                           4.0       2
                           5.0       2
                           6.0       2
                           7.0       2
TimeIncrement(Domain&)     1.0  

In [108]:
print(gfs[0].graph.roots[0].df_index)
a = special_lookup(gfs[0], gfs[0].graph.roots[0])
print(a)

-2445666782195161966


AttributeError: 'str' object has no attribute 'callpath'